# MNIST- handwritten digit recognition Part 2
In this modeule, we will talk about how we can further improve performance using various techniques.

## Batch Normalization
Do you remember we normalized input images such that they have zero mean? The training converges faster when images are normalized (zero mean and unit variance) and decorrelated. However, the parameter update during the training changes distributions in each layer, which is called *internal covariant shift*. Ioffe and Szegedy suggested [batch normalization](https://arxiv.org/abs/1502.03167) to normalize and decorrelate inputs to the mid-layers to resolve this issue and make the netwrok training converges faster. 

In [ ]:
# Implement Batch Normalization
import numpy
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.utils import to_categorical
from keras import backend as K

K.set_image_dim_ordering( 'tf' )

# fix random seed for reproducibility
seed = 123
numpy.random.seed(seed)
# load data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
# reshape to be [samples][width][height][channel]
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1).astype( 'float32' )
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1).astype( 'float32' )
# normalize inputs from 0-255 to 0-1
X_train = X_train / 255
X_test = X_test / 255
# one hot encode outputs
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
num_classes = y_test.shape[1]

def BN_model():
### YOUR TURN
    # Create a model with 4 convolutional layers (2 repeating VGG stype units) and 2 dense layers before the output
    # Use Batch Normalization for every conv and dense layers
    # Use dropout layers if you like
    # Use Adam optimizer

    return model

# build the model
model = BN_model()

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=20, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

From the above, can you get test error below 0.5%?

Where should you position the batch norm layer to implement the batch norm correctly?

**Claim:** Some people argue that they can get as good or better result by incorrectly implementing batchnorm such that the batchnorm comes after the activation layer. Test if this is true. What test wrror do you get?

In [ ]:
# Implement Batch Normalization - after the activation 

def BNr_model():
### YOUR TURN
    # Using the same architecture above, 
    # except that the orders of a batchnormalization layer and a activation layer are reversed, 
    # build a model and test if the claim above is true.

    return model
# build the model
model = BNr_model()

# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), nb_epoch=20, batch_size=200, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

### Recording loss and metric
The output of `model.fit` by default (in Keras 2) returns a dictionary of model history (also it can be called using the callback). The dictionary has keys loss and metric (when you specified the metric in the model.complie) for train and validation each. For our case here it would be: 'val_loss', 'val_acc', 'loss', 'acc'. A good use of such log is to monitor whether it's over fitting. When overfits, you will see the validation loss may go up at some point while train loss continues go down. Let's get rid of batch norm layers and run the model with higher running rate lr=0.01 and longer epoch (50) to see if it overfits (Answer: Yes it does, quite terribly).

In [ ]:
import time
from keras.optimizers import Adam

def model_overfit():
### YOUR TURN
    # 1) Create a model with the same architecture above (4 convs and 2 denses before output) and hyperparameters, 
    # but without any batch normalization and dropouts.
    # 2) To make this overfit surely, let's change the learning rate of our Adam optimizer. Set the learning rate to 0.01.
    # 3) After running the training, plot the train and validation accuracy using the model output hisoty.
    
    return model

# build the model
model = model_overfit()

# Fit the model
t0=time.time()
log = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=200, verbose=2)
t1=time.time()
print(t1-t0," seconds")

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("CNN Error: %.2f%%" % (100-scores[1]*100))

#### Tune Learning rate
Without inserting batchnorm or dropout again, decrease learning rate and run for 50 epochs, plot the accuracy from train and validation. What is the highest learning rate that it doesn't overfit? What is the validation accuracy as a result?

In [ ]:
#Your code here

#### Add Dropout
Now, add dropouts and run with the same hyperparameters (learning rate, epochs) you found from above. Time the model.fit() using `time.time`. 
1) Does it take longer training time by adding dropouts?
2) For the same epoch, is your final validation accuracy better? If not better and you're sure it's not overfitting yet, try to increase either your learning rate or epoch, OR change your dropout rate(s). Record your optimum values. 

In [ ]:
#Your code here

#### Add Batch Normalization
Now, get rid of dropouts and add batch normalization layers. Choose learning rate between 0.01 and 0.001. Find the largest learning rate that still does not overfit but gives highest accuracy.
Time model.fit() using `time.time`. 
Plot the 'acc' and 'val_acc'
Compare the learning rate with those from Exercise 1 and 2. What do you find?

In [ ]:
#Your code here

### Quiz.

#### 1. 
What are the advantages of a CNN over a fully connected ANN for image classificaion?

#### 2. 
Consider a CNN composed of 3 convolutional layers, each with 3x3 kernels, a stride of 2, and with 'same' padding. The first layer outputs a featuremap with 100 cahnnels, the second layer outputs a featuremap with 200 depth, and the last outputs one with 400 depth. The input is color (RGB) images of 200x300 pixels. What is the total number of parameters for this CNN model?

#### 3.
If your GPU runs out of memory while you train a CNN model, what can you do resolving the issue? List at least 3 ways to 